In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
#Dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()

RowNumber  CustomerId   Surname  ...  IsActiveMember EstimatedSalary Exited
0          1    15634602  Hargrave  ...               1       101348.88      1
1          2    15647311      Hill  ...               1       112542.58      0
2          3    15619304      Onio  ...               0       113931.57      1
3          4    15701354      Boni  ...               0        93826.63      0
4          5    15737888  Mitchell  ...               1        79084.10      0

[5 rows x 14 columns]

In [4]:
# Preprocessing the data
# Drop the columns - RowNumber, CustomerId, Surname
df = df.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)
df

CreditScore Geography  Gender  ...  IsActiveMember  EstimatedSalary  Exited
0             619    France  Female  ...               1        101348.88       1
1             608     Spain  Female  ...               1        112542.58       0
2             502    France  Female  ...               0        113931.57       1
3             699    France  Female  ...               0         93826.63       0
4             850     Spain  Female  ...               1         79084.10       0
...           ...       ...     ...  ...             ...              ...     ...
9995          771    France    Male  ...               0         96270.64       0
9996          516    France    Male  ...               1        101699.77       0
9997          709    France  Female  ...               1         42085.58       1
9998          772   Germany    Male  ...               0         92888.52       1
9999          792    France  Female  ...               0         38190.78       0

[10000 rows x 11 columns]

In [5]:
# Encoding for categorical variable
label_encoder_gender = LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])
df

CreditScore Geography  Gender  ...  IsActiveMember  EstimatedSalary  Exited
0             619    France       0  ...               1        101348.88       1
1             608     Spain       0  ...               1        112542.58       0
2             502    France       0  ...               0        113931.57       1
3             699    France       0  ...               0         93826.63       0
4             850     Spain       0  ...               1         79084.10       0
...           ...       ...     ...  ...             ...              ...     ...
9995          771    France       1  ...               0         96270.64       0
9996          516    France       1  ...               1        101699.77       0
9997          709    France       0  ...               1         42085.58       1
9998          772   Germany       1  ...               0         92888.52       1
9999          792    France       0  ...               0         38190.78       0

[10000 rows x 11 columns]

Gender - Male and Female converted to 0s and 1s.

In [6]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [8]:
# Geographical feature has 3 countries
# Can't use label encoder as it will assign 0, 1, 2 so ANN might consider one country over other in terms of priority and model will fail. Better to use OneHotEncoding
from sklearn.preprocessing import OneHotEncoder
ohe_geo= OneHotEncoder()
geo_encoder = ohe_geo.fit_transform(df[['Geography']])
geo_encoder


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [11]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
ohe_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [12]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [13]:
# combine the above to the original data
df = pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df.head()

CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0          619       0  ...                0.0              0.0
1          608       0  ...                0.0              1.0
2          502       0  ...                0.0              0.0
3          699       0  ...                0.0              0.0
4          850       0  ...                0.0              1.0

[5 rows x 13 columns]

In [14]:
# saving the encoder and stdscaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('ohe_geo.pkl','wb') as file:
    pickle.dump(ohe_geo,file)



In [15]:
df.head()

CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0          619       0  ...                0.0              0.0
1          608       0  ...                0.0              1.0
2          502       0  ...                0.0              0.0
3          699       0  ...                0.0              0.0
4          850       0  ...                0.0              1.0

[5 rows x 13 columns]

In [16]:
# Divide the dataset into Independent and Dependent features.
X = df.drop('Exited',axis=1)
y=df['Exited']

# Split data into trg and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Scaling the features - StdScaler()
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [18]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)
    


In [19]:
df

CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0             619       0  ...                0.0              0.0
1             608       0  ...                0.0              1.0
2             502       0  ...                0.0              0.0
3             699       0  ...                0.0              0.0
4             850       0  ...                0.0              1.0
...           ...     ...  ...                ...              ...
9995          771       1  ...                0.0              0.0
9996          516       1  ...                0.0              0.0
9997          709       0  ...                0.0              0.0
9998          772       1  ...                1.0              0.0
9999          792       0  ...                0.0              0.0

[10000 rows x 13 columns]

### ANN Implementation

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


In [21]:
# Build ANN model
# Creating sequential model
model = Sequential([
    # Layers 
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1
    Dense(32,activation='relu'), #HiddenLayer(HL)2
    Dense(1, activation='sigmoid') #o/p layer
    # 64 neurons
])

/Users/dhruv/Developer/DeepLearning_Project/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

Parameters (Param) are the combination of both bias and weights.

In [23]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
# here adam optimizer's learning rate can be set.
loss=tf.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x30db560c0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [24]:
# Compile the model
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
# if we use optimizer='adam' this has the fixed learning rate

In [36]:
# Setting up tensorboard

log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [30]:
# Setting up Earlystopping
# We can train a model for any number of epochs.
# Say after x number of epochs model is varying a very little amt. then we must stop the trg. for efficient usage of resource.

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)


In [31]:
# Training The Model
history =model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 100, callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - accuracy: 0.8667 - loss: 0.3225 - val_accuracy: 0.8585 - val_loss: 0.3451
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step - accuracy: 0.8655 - loss: 0.3334 - val_accuracy: 0.8570 - val_loss: 0.3529
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - accuracy: 0.8609 - loss: 0.3312 - val_accuracy: 0.8590 - val_loss: 0.3496
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.8676 - loss: 0.3257 - val_accuracy: 0.8585 - val_loss: 0.3507
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - accuracy: 0.8709 - loss: 0.3178 - val_accuracy: 0.8640 - val_loss: 0.3380
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - accuracy: 0.8653 - loss: 0.3180 - val_accuracy: 0.8580 - val_loss: 0.3560
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step - accuracy: 0.8812 - loss: 0.3103 - val_accuracy: 0.8630 - val_loss: 0.3502
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - accuracy: 0.8725 - loss: 0

In [32]:
model.save('model.h5')
# h5 file is compatible with keras

In [33]:
# Loading tensorboard extension
%load_ext tensorboard

In [35]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 31235), started 0:05:45 ago. (Use '!kill 31235' to kill it.)

### Predictions

In [37]:
# Load the pickle file
# Refer: prediction.ipynb
